## RegressionSklienModel

In [1]:
import os, sys

sys.path.insert(0, os.path.abspath("../../.."))

In [2]:
from models.xgboost import RegressionSklienModel
import pandas as pd
from pandas.plotting import scatter_matrix
from lib.data_splitter import split_features_and_target_into_train_and_test_set
from lib.data_util import load_candlesticks
import xgboost as xgb
from sklearn.metrics import mean_squared_error, make_scorer
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV

In [3]:
candlesticks = load_candlesticks("BTCUSDT", "1h", custom_data_path="../../../tmp")

candlesticks

Closetime of newest candle is 1587038399999
Only using data on file. Will not download new data from Binance.


,open time,open,high,low,close,volume,close time,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume
0,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181009,1502945999999,2.023661e+05,171,35.160503,1.509525e+05
1,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234916,1502949599999,1.003048e+05,102,21.448071,9.260828e+04
2,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229691,1502953199999,3.128231e+04,36,4.802861,2.079532e+04
3,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443249,1502956799999,1.924106e+04,25,2.602292,1.129135e+04
4,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807,1502960399999,4.239504e+03,28,0.814655,3.552747e+03
...,...,...,...,...,...,...,...,...,...,...,...
23252,1587020400000,6688.63,6818.46,6686.39,6780.00,3399.164609,1587023999999,2.300670e+07,22202,1901.518707,1.285952e+07
23253,1587024000000,6893.24,6923.52,6871.13,6888.30,1641.824781,1587027599999,1.132671e+07,12148,873.070968,6.024127e+06
23254,1587027600000,7045.30,7097.00,7017.00,7027.09,4302.835917,1587031199999,3.038567e+07,31923,2361.179694,1.667550e+07
23255,1587031200000,7027.70,7061.80,7026.77,7040.00,1512.420788,1587034799999,1.066239e+07,12131,938.969209,6.619599e+06


In [4]:
features = RegressionSklienModel.generate_features(candlesticks, pd.DataFrame(index=candlesticks.index))
target = RegressionSklienModel.generate_target(candlesticks, features)

features.head(2000)

,open,high,low,close,volume,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume,volume_adi,...,trend-macd_signal-15_5_3,momentum_roc-2,momentum_roc-3,momentum_roc-5,momentum_roc-10,momentum_roc-15,momentum_roc-20,momentum_roc-30,momentum_roc-50,momentum_roc-80
0,4261.48,4313.62,4261.32,4308.83,47.181009,2.023661e+05,171,35.160503,1.509525e+05,38.538675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102,21.448071,9.260828e+04,45.125637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36,4.802861,2.079532e+04,43.899314,...,NaN,0.360191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25,2.602292,1.129135e+04,48.342563,...,NaN,0.803417,0.955248,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28,0.814655,3.552747e+03,48.565612,...,NaN,0.840358,1.051370,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,7438.00,7485.01,7425.00,7460.00,97.914733,7.288022e+05,735,21.834075,1.627530e+05,3324.498668,...,80.073913,0.810811,0.865333,1.842586,1.773533,5.605291,5.815603,4.030121,0.811083,0.134631
1996,7460.00,7489.99,7428.25,7442.38,52.839960,3.943658e+05,640,22.464537,1.677127e+05,3295.844929,...,79.751422,0.166622,0.572703,1.397171,1.147191,5.267044,5.419432,4.138023,1.598712,0.590510
1997,7468.99,7468.99,7352.51,7414.81,86.595162,6.407901e+05,816,35.861318,2.656635e+05,3301.881611,...,73.850030,-0.605764,-0.204441,0.254327,0.608278,1.921787,5.654175,3.979655,1.713048,0.328393
1998,7417.70,7770.02,7352.16,7499.87,361.347742,2.731759e+06,2552,206.895858,1.564692e+06,3196.000652,...,74.995736,0.772468,0.534450,1.349595,2.695601,2.766241,6.486866,4.738742,3.703524,1.119070


In [5]:
target.describe()

count    23257.000000
mean         0.149374
std          2.627953
min         -5.000000
25%         -2.000000
50%          0.000000
75%          2.000000
max          5.000000
Name: target, dtype: float64

In [12]:
(
    training_set_features,
    training_set_targets,
    test_set_features,
    test_set_targets,
) = split_features_and_target_into_train_and_test_set(features, {0: target}, 20)

In [7]:
# scatte = scatter_matrix(test_set_features.iloc[:, -5:], c=test_set_target.iloc[:], s=40, hist_kwds={"bins": 15}, figsize=(20,20))

In [14]:
raw_model = xgb.XGBRegressor(objective="reg:squarederror")

param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.9),
              'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 25],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4, 5, 10]
             }
rmse = make_scorer(mean_squared_error, greater_is_better=False)

r = RandomizedSearchCV(raw_model, param_distributions=param_dist,
                       scoring=rmse,
                       n_iter=3,
                       n_jobs=2)
r.fit(training_set_features, training_set_targets[0])

r.score(test_set_features, test_set_targets[0])


-9.433460595245597

In [15]:
r.best_params_


{'colsample_bytree': 0.7093174849510205,
 'learning_rate': 0.4729792056798079,
 'max_depth': 13,
 'min_child_weight': 2,
 'n_estimators': 937,
 'subsample': 0.9666213133434363}